In [1]:
# Author: Lai Yoke Yau
# Created At: 13/12/2024

from py2neo import Graph
import numpy as np
from sklearn.cluster import KMeans
import networkx as nx
from pyvis.network import Network

# Connect to Neo4j database (Replace with your connection details)
graph = Graph("neo4j+s://3a258465.databases.neo4j.io", auth=("neo4j", "qqFwDFq89rQO0lsvVIpOZGKU6O0L0bqQ5HruDuKOQ1g"))

ModuleNotFoundError: No module named 'networkx'

In [2]:
# Prompt the user to choose between synonyms or antonyms
relationship = input("Enter the relationship to query (synonym/antonym/hypernym/meronym): ").strip().lower()

# Validate input
if relationship not in ["synonym", "antonym", "hypernym", "meronym"]:
    print("Invalid input. Please enter 'synonym' or 'antonym' or 'hypernym' or 'meronym'.")
else:
    # Construct the query based on user input
    query = f"""
    MATCH (w:Word)-[:{relationship.upper()}]->(related:Word)
    RETURN w.name AS word, COLLECT(related.name) AS related_words
    LIMIT 5
    """
    # Execute the query
    result = graph.run(query)
    
    # Print the results
    for record in result:
        print(f"Word: {record['word']}, Related Words ({relationship}): {record['related_words']}")


Enter the relationship to query (synonym/antonym/hypernym/meronym):  meronym


Word: sistem, Related Words (meronym): ['komponen']
Word: huruf, Related Words (meronym): ['aksara']
Word: abode, Related Words (meronym): ['rumah', 'kediaman']
Word: rumah, Related Words (meronym): ['bilik']
Word: tempat, Related Words (meronym): ['budaya', 'bahasa']


In [3]:
# Create an empty NetworkX graph
G = nx.Graph()

# Query to get nodes and their relationships
query = f"""
MATCH (w:Word)-[:{relationship.upper()}]->(related:Word)
RETURN w.name AS word, related.name AS related_word
"""
result = graph.run(query)
    
# Add nodes and edges to the NetworkX graph
for record in result:
    word = record["word"]
    related_word = record["related_word"]
    G.add_edge(word, related_word)
    
# Output some basic graph stats
print(f"Relationship type: {relationship.capitalize()}")
print(f"Number of nodes: {len(G.nodes())}")
print(f"Number of edges: {len(G.edges())}")


Relationship type: Meronym
Number of nodes: 1467
Number of edges: 1921


In [4]:
# Create node embeddings using node degrees
node_embeddings = np.array([G.degree(node) for node in G.nodes()])

# Apply K-means clustering
kmeans = KMeans(n_clusters=3)
labels = kmeans.fit_predict(node_embeddings.reshape(-1, 1))  # Reshaping for K-means

# Show only the first 5 nodes
for node, label in zip(list(G.nodes())[:5], labels[:5]):  # Limiting to the first 5 nodes
    print(f"Node: {node}, Cluster: {label}")


Node: sistem, Cluster: 0
Node: komponen, Cluster: 2
Node: huruf, Cluster: 2
Node: aksara, Cluster: 0
Node: abode, Cluster: 0


In [5]:
# Save the NetworkX graph to a file
nx.write_gml(G, "graph.gml")

# Load a graph from a file
G_loaded = nx.read_gml("graph.gml")


In [6]:
# Create a Pyvis network object
net = Network(notebook=True)

# Load the graph into Pyvis
net.from_nx(G_loaded)

# Show the graph (interactive)
net.show("graph.html")


graph.html
